In [3]:
import pandas as pd
import numpy as np


In [2]:
df = pd.read_csv('../pitches.csv')
#drop unkown prediction classes and prediction classes with no definition from data source
df.drop_duplicates(inplace=True)
df = df[df.pitch_type != 'UN']
df = df[df.pitch_type != 'FA']

features = ['break_angle', 'break_length', 'end_speed', 'start_speed', 'spin_rate', 'pitch_type']
#remove wild pitches or intentional walks as break length outliers. Code sourced from https://stackoverflow.com/questions/23199796/detect-and-exclude-outliers-in-pandas-data-frame
for feature in features:
    if feature != 'pitch_type':
        df = df[np.abs(df[feature]-df[feature].mean()) <= (12*df[feature].std())]

#drop data points where prediction class is unreliable
df = df[df.type_confidence >= 1.0]

#Drop any data with incomplete records
df.dropna(inplace=True)

In [2]:
from scipy.special import expit
import sys
import pandas as pd
import numpy as np

class MultiLayerPerceptronBase(object):
    def __init__(self, layers = 2, layer_width=[4, 3, 3], C = 0.0,
                 epochs = 500, eta = 0.001, random_state = None):
        np.random.seed(random_state)
        self.C = C
        self.epochs = epochs
        self.eta = eta
        self.layer_width = layer_width
        #add widths for input and output layers
#         self.layer_width.insert(0, 8)
        #output layer
#         self.layer_width.append(8)
        
        
        
    @staticmethod
    def _encode_labels(y):
        onehot = pd.get_dummies(y).values.T
        return onehot
    
    def _initialize_weights(self):
        self.weights = []
        for idx, w in enumerate(self.layer_width[:-1]):
            Wn_num_elements = (w + 1) * self.layer_width[idx+1]
            self.weights.append(np.random.uniform(-1.0, 1.0, size = Wn_num_elements).
                                reshape(self.layer_width[idx+1], w + 1))
#         for w in self.weights:
#             print(w.shape)
    
    @staticmethod
    def _sigmoid(z):
        return expit(z)
    
    @staticmethod
    def _add_bias_unit(X, how='column'):
        """Add bias unit (column or row of 1s) to array at index 0"""
        if how == 'column':
            ones = np.ones((X.shape[0], 1))
            X_new = np.hstack((ones, X))
        elif how == 'row':
        
            ones = np.ones((1, X.shape[1]))
            X_new = np.vstack((ones, X))
        return X_new
    
    @staticmethod
    def _l2_reg(lambda_, weights):
        t = 0
        for w in weights:
            t += np.mean(x[:, 1:] **2)
        t = np.sqrt(t)
        t *= (lambda_/len(weights))
        return t
    def _cost(self, A_n, y_enc, weights):
        cost = np.mean((Y_enc - A_n) **2)
        L2_term = self._l2_reg(self.C, weights)
        return cost + L2_term
    

In [3]:
class MultiLayerPerceptron(MultiLayerPerceptronBase):
    def _feedforward(self, X):
        A = []
        Z = []
        for idx, W_n in enumerate(self.weights):
            if len(Z) > 0:
                A_n = self._sigmoid(Z[-1])
                A_n = self._add_bias_unit(A_n.reshape(-1, 1), how="row")
                Z_n = W_n @ A_n
                A.append(A_n)
                Z.append(Z_n)
            else:
                A_n = self._add_bias_unit(X.T.reshape(-1, 1), how="row")
                Z_n = W_n @ A_n
                A.append(A_n)
                Z.append(Z_n)
        return A, Z
    
    def _get_gradient(self, A, Z, Y_enc):
        gradients = []
        for w in self.weights:
            gradients.append(np.zeros(w.shape))
            
            
        

In [8]:
x = np.array([1,2,3,4,5,6,7,8])
x.reshape(1,8)
mc = MultiLayerPerceptron(4, [x.shape[0], 4, 2,3,4])
mc._initialize_weights()
A, Z = mc._feedforward(x)
print(A)



[array([[1.],
       [1.],
       [2.],
       [3.],
       [4.],
       [5.],
       [6.],
       [7.],
       [8.]]), array([[1.00000000e+00],
       [6.97579406e-09],
       [7.14217397e-01],
       [3.32141668e-06],
       [1.19518322e-03]]), array([[1.        ],
       [0.7334974 ],
       [0.63339697]]), array([[1.        ],
       [0.53652242],
       [0.6495008 ],
       [0.28365213]])]
